In [ ]:
from seismic_purifier import RepresentationLearningSingleAutoencoder, RepresentationLearningDenoisingSingleAutoencoder, RepresentationLearningMultipleAutoencoder
from kfold_trainer import KfoldTrainer
from config import KFOLD_SPLITS

In [ ]:
# Experiment name.
EXP_NAME = "exp_test"

# Should be one of the RepresentationLearningSingleAutoencoder, RepresentationLearningDenoisingSingleAutoencoder, RepresentationLearningMultipleAutoencoder
MODEL_CLASSES = [RepresentationLearningSingleAutoencoder, RepresentationLearningDenoisingSingleAutoencoder, RepresentationLearningMultipleAutoencoder]

# Should be stead or instance.
DATASETS = ["instance", "stead"]

# Number of epochs
NUM_EPOCHS = 10

In [ ]:
# For all splits, train the model over defined datasets.
for train_dataset in DATASETS:
    for model_class in MODEL_CLASSES:
        for split in range(KFOLD_SPLITS):
            kfold_trainer = KfoldTrainer(
                EXP_NAME, model_class, train_dataset, split, epochs=NUM_EPOCHS
            )
            kfold_trainer.train()